In [1]:
import tinny
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data_train = np.array(pd.read_csv('./data/classification/train.csv')).T
data_test = np.array(pd.read_csv('./data/classification/test.csv')).T

m, n = data_train.shape

X_train = data_train[1:n]
y_train = data_train[0]

X_train, X_test, y_train, y_test = train_test_split(X_train.T, y_train, train_size=35000, random_state=42)

X_train = X_train.T / 255.0
X_test = X_test.T / 255.0

In [3]:
l1 = tinny.DenseLayer(784, 10, "ReLU")
l2 = tinny.OutputLayer(10, 10, "softmax")
nn = tinny.TiNNyNetwork(problem_type="classification", loss_function="crossentropy", layers=[l1, l2])

nn.train(X_train, y_train, iterations=350, learning_rate=0.01)
nn.test(X_test, y_test)

Iteration 0
Loss: 0.9893994233252734
Accuracy: 0.10005714285714286
Iteration 10
Loss: 5.097589151191328
Accuracy: 0.44817142857142855
Iteration 20
Loss: 3.0601543617899707
Accuracy: 0.5970285714285715
Iteration 30
Loss: 2.5948627368181714
Accuracy: 0.6772571428571429
Iteration 40
Loss: 2.3060338513892673
Accuracy: 0.6678285714285714
Iteration 50
Loss: 2.1379169363650035
Accuracy: 0.7278857142857142
Iteration 60
Loss: 2.1845288074683986
Accuracy: 0.7658571428571429
Iteration 70
Loss: 2.191593896063989
Accuracy: 0.7889428571428572
Iteration 80
Loss: 2.185876047591833
Accuracy: 0.8029428571428572
Iteration 90
Loss: 2.2072042307405115
Accuracy: 0.817
Iteration 100
Loss: 2.204751743282092
Accuracy: 0.8275714285714286
Iteration 110
Loss: 2.1975741000843216
Accuracy: 0.8356571428571429
Iteration 120
Loss: 2.208538380158795
Accuracy: 0.8440857142857143
Iteration 130
Loss: 2.2018124119656717
Accuracy: 0.8501714285714286
Iteration 140
Loss: 2.1993015520302444
Accuracy: 0.8558857142857142
Iterati